In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 人力资源数据分析

1. **Question 提出问题** (对数据集提问的常见思路：)
      * 最高/最多 - 这些最受欢迎/最热门/最高频率的选项：最常见的辞职理由是啥？
      * 最少/最低 - 可能正是要解决的痛点和短板！：哪个月份入职最少？ 
      * 趋势/模式 - 随着时间发展的趋势或者模式；可以是横向（广度）或者纵向（深度）的趋势比较
      * 对比 - 没有对比就没有分析：哪支球队在这段时间表现的其它球队更好？
      * 相关性 - 有哪些最重要的因素能够帮助我们得出结论或者进行预测?
      
      

2. **Wrangle 整理数据** (为了获得高质量的干净的数据集，一般会通过三步来获取所需数据)

    * 收集 Gather : 收集所需的数据来回答你的问题
    * 评估 Assess : 评估数据集的数据质量或结构中存在的任何问题
    * 清理 Clean : 通过修改、替换或删除数据来清理数据，以确保你的数据集具有较高的质量和良好的结构。

`整理数据` 一般是指收集 - 评估 - 清理数据的过程，但是你在清理数据时所做的修改，只是使数据集可以被分析，并不会优化你分析、可视化界面或者模型。

3. Explore 探索性数据分析（EDA）

    * `探索性数据分析` 则意味着你要探索并增加数据, 从而将数据分析、可视化和模型的潜力最大化。比如，删除异常值可以在可视化中更凸显分布的核心等。

# 提出问题：

1. 入职的高峰期是什么时候？
2. 职场的男女比例
3. 最常见的被辞退的原因是什么？
4. 最常见被辞退的年纪
5. 他们通常都在哪个部门？
6. 员工的来源

In [ ]:
import numpy as np 
import pandas as pd 

import time 

In [ ]:
df = pd.read_csv("../input/human-resources-data-set/core_dataset.csv")
display(df.shape)
display(df.head())

In [ ]:
display(df.columns)

** 员工姓名
* 员工编号
* 州
* 邮编
* 出生日期
* 年龄
* 性别
* 婚姻状况
* 公民资格
* 西班牙裔/拉丁裔
* RaceDesc
* 雇用日期
* 终止日期
* 期限理由
* 就业状况
* 部门
* 职位
* 工资率
* 经理姓名
* 员工来源
* 绩效得分*

In [ ]:
df['DOB'] = pd.to_datetime(df['DOB'])

In [ ]:
df["Date of Hire"] = pd.to_datetime(df["Date of Hire"])

In [ ]:
df.shape

## 1. 入职的高峰期是什么时候？

In [ ]:
import seaborn as sns

In [ ]:
m=[i.month for i in df["Date of Hire"]]

In [ ]:
df_m = pd.DataFrame(m)
df_m.columns = ["month"]
df_m = df_m["month"]
df_m.value_counts().plot(kind = "bar", figsize = (10, 8))

* 一月份跟七月份是入职的高峰期，在美国过完圣诞假期跟国庆日之后也会想跳槽吗？

## 2. 职场的男女比例

In [ ]:
df["Sex"].value_counts()

看起来不足以说明女生入职比男生容易，只能说在这几年中在职的女性工作者比男性工作者多

## 3. 最常见的被辞退的原因是什么？

In [ ]:
# 首先去掉所有还在公司的数据

df_tr = df[df["Reason For Term"] != "N/A - still employed"]
df_tr["Reason For Term"].value_counts().plot(kind = "bar", figsize = (10, 8))

排名第一名的是『其他原因』这部分比较主观，我们主要是看客观原因

比较意外的是：往后是『做得不开心』『钱太少』

看样子这方面跟国内也是一样的

## 被辭退的年紀

這邊我們練習的是 `pivot_table` 方法，這在 pandas 當中是非常實用的。
俗称透视表
在我们想要分组查询的时候很有帮助，如果已经熟悉  `groupby` 的小伙伴會發現他們很相似。
但是 `groupby` 需要在最后再进行一个操作，`pivot_table`则是单纯的展示结果。

> 使用方法 `pd.pivot_table(数据, index=['要聚合的 index'])`

In [ ]:
pd.pivot_table(df,index=["Reason For Term"])

这边可以看到
『做得不开心』『钱不够』被辞退的年纪是四十岁，看起来可能跟中年危机有关 :)



## 4. 他们都在哪个部门？

这边可以练习代码的简化，用炼式的方式往后写可以清晰地表达。

In [ ]:
df["Department"].value_counts().plot(kind = "bar", figsize = (10, 8))

这边可以看到员工大多都在生产部门，这是整体企业在转型的迹象。

**近一步的好奇探索了一下，每个行业他们的年纪是怎么样？**

In [ ]:
pd.pivot_table(df,index=["Department"])

值得注意的是在软件工程师到了 51 岁都有人，而 IT 企业主要的是 37 岁。
看起来自己还是太年轻啊（茶）

## 员工来源

这边我们探索一下，公司的员工主要来源是什么？
校招？社招？内推？

In [ ]:
df["Employee Source"].value_counts().plot(kind = "bar", figsize = (10, 8))

看出来排名第一名的是 员工内推

## 小節：

以上是我们对于 HR 数据集所做的探索性分析(EDA)，在这里我们往往会抱着理所当然的结果去看数据。
而在結果出來之後會用理所當然的方式去解釋結論， 這是很危險的事情。


1. 入职的高峰期是什么时候：
    > 一月，七月（但是不能因此断定就是假期后跳槽）
2. 职场的男女比例：
    > 女性在职人数比男性多
3. 最常见的被辞退的原因是什么？
    > 做得不开心，钱太少
4. 最常见被辞退的年纪
    > 因为做的不开心，还有钱太少辞职的员工大多是在 40 岁
5. 他们通常都在哪个部门？
    > 生产部门
    * 5.1 软件行业的年纪是？
        > 很惊讶地发现到 51 岁都有
6. 员工的来源
    > 员工推荐
    
    
##  NEXT:

我们的下一步是根据其他的表格数据，进行对于绩效评分的分類。
